In [1]:
import torch
import json_repair
import pandas as pd

from tqdm import tqdm
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    #model_name="unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.003 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## data

In [4]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_dataset_pg.csv")

In [5]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f..."
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],..."
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]"
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]"
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur..."
...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [..."
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']..."
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [..."
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr..."


In [6]:
def get_nr_emotions(row):

    return len(row.emotion_c)

In [7]:
df['nr_emotions'] = df.apply(lambda row: get_nr_emotions(row), axis=1)

In [8]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c,nr_emotions
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",137
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",99
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",40
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",22
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",83
...,...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",166
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",98
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",143
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",144


In [9]:
df.nr_emotions.value_counts()

nr_emotions
116    11
44     10
156    10
123     9
109     9
       ..
313     1
236     1
256     1
187     1
233     1
Name: count, Length: 252, dtype: int64

In [10]:
file_names_l = df.file_name.unique().tolist()

In [11]:
comics_titles = [
    
    "Nightwing",
    "Worlds Without a Justice League - Green Lantern",
    "Human Target",
    "American Vampire",
    "Dragon Age",
    "Dragon Age",
    "The Walking Dead",
    "Worlds Without a Justice League - Green Lantern",
    "Dark Crisis: The Flash",
    "Danger Street",
    "Tiny Titans",
    "The Amazing Adventures of the Ninja Turtles",
    "Sonic The Hedgehog",
    "Love Everlasting",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "Fantasmas",
    "American Vampire",
    "American Vampire",
    "Dragon Age",
    "Stillwater",
    "Jurassic League",
    "John Carpenter's Tales for a Halloweenight",
    "DC vs. Vampires",
    "Thief Of Thieves",
    "The Walking Dead",
    "The Walking Dead",
    "The Walking Dead",

]


In [12]:
df['comics_title'] = df['file_name'].apply(lambda x: comics_titles[file_names_l.index(x)] if x in file_names_l else None)

In [13]:
df_train = df[df.split == "TRAIN"].reset_index(drop=True)

In [14]:
len(df_train)

718

### Comics dataset

In [15]:
def build_instruction():
   
    emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

INPUT:
- You will receive a list of utterances from a page in a comic book
- The utterance may express one or multiple emotions

TASK:
1. Carefully analyze the emotional context and tone of each utterance in the page
2. Identify applicable emotions from the following classes:
   {formatted_classes}
3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.

RULES:
1. Use ONLY the labels listed above
2. Output must be a JSON with single key "page_utterance_emotions"
3. Value must be an array where:
   - Each element is an array of emotions for one utterance
   - Order matches the input utterances order
   - Multiple emotions are allowed per utterance
4. No explanations, only JSON output

IMPORTANT:
- Each array element corresponds to one utterance
- One utterance can have multiple emotions
- Maintain exact spelling and case of emotion labels
- Keep emotions in arrays even for single emotions

"""
    return instruction

In [16]:
# def build_instruction():
  
#     emotion_classes = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Emotion Analysis Expert Role

# You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content while considering conversational context.

# INPUT:
# - You will receive a list of 6 consecutive utterances from a comic book:
#   * The first 5 utterances provide conversational context
#   * The 6th (last) utterance is the one to be classified
# - Each utterance may express one or multiple emotions

# TASK:
# 1. Read through the context utterances to understand the emotional flow
# 2. Carefully analyze the emotional context, tone, and potential emotional shifts in the final utterance
# 3. Identify applicable emotions for the final utterance only from the following classes:
#    {formatted_classes}

# OUTPUT REQUIREMENTS:
# - Format: JSON object with a single key "list_emotion_classes"
# - Value: Array of one or more emotion classes as strings
# - The classification should be for the final utterance only, using previous utterances as context

# IMPORTANT NOTES:
# - Do not include any explanations in the output, only the JSON object
# - Use the context to better understand emotional transitions and current emotional state
# - Context utterances may reveal emotional buildup or shifts that influence the final utterance

# """
#     return instruction

In [17]:
instruction = build_instruction()

In [18]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df_train.iterrows():
        
        #prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
        #human_msg = {'from': 'human', 'value': prompt}
        utterances_l = eval(row['utterance'])
        usr_prompt = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        sys_msg = {'role': 'system', 'content': instruction}
        user_msg = {'role': 'user', 'content': "Here is the list of utterances that you will classify: \n" + usr_prompt}

        #obj = {"list_emotion_classes": row['emotion_u']}
        #obj = row['emotion_u']
        #assistant_msg = {'from': 'gpt', 'value': obj}
        assistant_msg = {'role': 'assistant', 'content': f'{{"list_emotion_classes": {row["emotion_c"]}}}'}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [19]:
comics_dataset = []

for i in range(len(sys_msg_l)):
    
    #obj = {"list_emotion_classes": ["Anger", "Fear"]}

    #comics_dataset.append([human_msg_l[i], assistant_msg_l[i]])
    comics_dataset.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [20]:
len(comics_dataset)

718

In [21]:
comics_dataset[0]

[{'role': 'system',
  'content': '### Emotion Analysis Expert Role\n\nYou are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.\n\nINPUT:\n- You will receive a list of utterances from a page in a comic book\n- The utterance may express one or multiple emotions\n\nTASK:\n1. Carefully analyze the emotional context and tone of each utterance in the page\n2. Identify applicable emotions from the following classes:\n   "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"\n3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.\n\nRULES:\n1. Use ONLY the labels listed above\n2. Output must be a JSON with single key "page_utterance_emotions"\n3. Value must be an array where:\n   - Each element is an array of emotions for one utterance\n   - Order matches the input utterances order\n   - Multiple emotions are 

In [22]:
def fix_comics_dataset(comics_dataset):
    fixed_comics_dataset = []
    for conversation in comics_dataset:
        fixed_conversation = []
        for message in conversation:
            if isinstance(message['content'], list):  # If the 'value' is a list of emotions
                message['content'] = ', '.join(message['content'])  # Join the list into a string
            fixed_conversation.append(message)
        fixed_comics_dataset.append(fixed_conversation)
    return fixed_comics_dataset

In [23]:
fixed_comics_dataset = fix_comics_dataset(comics_dataset)

In [24]:
dataset = Dataset.from_dict({
    'conversations': fixed_comics_dataset
})

In [25]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 718
})

In [26]:
dataset[0]['conversations']

[{'content': '### Emotion Analysis Expert Role\n\nYou are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.\n\nINPUT:\n- You will receive a list of utterances from a page in a comic book\n- The utterance may express one or multiple emotions\n\nTASK:\n1. Carefully analyze the emotional context and tone of each utterance in the page\n2. Identify applicable emotions from the following classes:\n   "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"\n3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.\n\nRULES:\n1. Use ONLY the labels listed above\n2. Output must be a JSON with single key "page_utterance_emotions"\n3. Value must be an array where:\n   - Each element is an array of emotions for one utterance\n   - Order matches the input utterances order\n   - Multiple emotions are allowed per utteranc

In [27]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template_comics(examples):
    messages = examples["conversations"]
    #messages = examples['input'] + examples['output']
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.


In [28]:
comics_dataset = dataset.map(apply_template_comics, batched=True)

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

In [29]:
comics_dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 718
})

In [30]:
def split_dataset(dataset, train_ratio=0.8):
    train_test = dataset.train_test_split(test_size=1 - train_ratio)
    return train_test

dataset_split = split_dataset(comics_dataset)

In [31]:
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

In [32]:
train_dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 574
})

In [33]:
print(train_dataset[452]['text'])

<|im_start|>system
### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

INPUT:
- You will receive a list of utterances from a page in a comic book
- The utterance may express one or multiple emotions

TASK:
1. Carefully analyze the emotional context and tone of each utterance in the page
2. Identify applicable emotions from the following classes:
   "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"
3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.

RULES:
1. Use ONLY the labels listed above
2. Output must be a JSON with single key "page_utterance_emotions"
3. Value must be an array where:
   - Each element is an array of emotions for one utterance
   - Order matches the input utterances order
   - Multiple emotions are allowed per utterance
4. No explana

In [34]:
OUTPUT_DIR = "/Utilisateurs/umushtaq/emotion_analysis_comics/outputs_dir_tmp"

In [35]:
args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="cosine",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        num_train_epochs=20,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        
        eval_strategy="steps",  # Run evaluation during training (can also use "epoch")
        eval_steps=25,  # Perform evaluation every 50 steps
        save_strategy="steps",  # Save the model every few steps
        save_steps=25,  # Save every 200 steps
        load_best_model_at_end=True,
    
        output_dir=OUTPUT_DIR,
        seed=0,
    )

In [36]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Replace with your train dataset
    eval_dataset=eval_dataset, 
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=args,
)

Map (num_proc=2):   0%|          | 0/574 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/144 [00:00<?, ? examples/s]

In [37]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 574 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 2
\        /    Total batch size = 32 | Total steps = 360
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
25,No log,0.639348
50,No log,0.650347
75,No log,0.834895
100,1.111600,0.903071
125,1.111600,0.977269
150,1.111600,1.119208
175,1.111600,1.171668
200,0.086500,1.198864
225,0.086500,1.257800
250,0.086500,1.301916


TrainOutput(global_step=360, training_loss=0.3464935249752469, metrics={'train_runtime': 1518.1827, 'train_samples_per_second': 7.562, 'train_steps_per_second': 0.237, 'total_flos': 3.766517906428723e+17, 'train_loss': 0.3464935249752469, 'epoch': 20.0})

In [38]:
model = FastLanguageModel.for_inference(model)

In [39]:
df_test = df[df.split == "TEST"].reset_index(drop=True)

In [40]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df_test.iterrows():
        
        #prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
        #human_msg = {'from': 'human', 'value': prompt}
        utterances_l = eval(row['utterance'])
        usr_prompt = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        sys_msg = {'role': 'system', 'content': instruction}
        user_msg = {'role': 'user', 'content': "Here is the list of utterances that you will classify: \n" + usr_prompt}

        #obj = {"list_emotion_classes": row['emotion_u']}
        #obj = row['emotion_u']
        #assistant_msg = {'from': 'gpt', 'value': obj}
        assistant_msg = {'role': 'assistant', 'content': ""}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [41]:
test_messages = []

for i in range(len(sys_msg_l)):
    
    #obj = {"list_emotion_classes": ["Anger", "Fear"]}

    #comics_dataset.append([human_msg_l[i], assistant_msg_l[i]])
    test_messages.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [42]:
# human_msg_l = []
# assistant_msg_l = []

# for _, row in df_test.iterrows():
        
#         prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
#         human_msg = {'role': 'user', 'content': prompt}
        
#         #obj = {"list_emotion_classes": row['emotion_u']}
#         obj = row['emotion_u']
#         assistant_msg = {'role': 'assistant', 'content': ""}
        
#         human_msg_l.append(human_msg)
#         assistant_msg_l.append(assistant_msg)
        
        

In [43]:
# test_messages = []

# for i in range(len(human_msg_l)):
    
#     #obj = {"list_emotion_classes": ["Anger", "Fear"]}

#     test_messages.append([human_msg_l[i], assistant_msg_l[i]])

In [44]:
len(test_messages)

156

In [45]:
# test_messages = test_messages[:100]

In [46]:
raw_outputs = []

for message in tqdm(test_messages):
    
    inputs = tokenizer.apply_chat_template(
    message,
    tokenize=True,
    add_generation_prompt=True,
    #return_dict=True,
    return_tensors="pt",
).to("cuda")
    #print(inputs)
    #break
    
    output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
    #output = model.generate(**inputs, max_new_tokens=128)[0]
    
    input_length = inputs.shape[1]
    generated_tokens = output[input_length:]
    
    decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
    #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
    raw_outputs.append(decoded_output)
    #break

100%|██████████| 156/156 [03:59<00:00,  1.53s/it]


In [47]:
#print(tokenizer.decode(inputs[0]))

In [76]:
len(raw_outputs)

156

In [77]:
raw_outputs

['{"list_emotion_classes": [[\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\']]}',
 '{"list_emotion_classes": [[\'joy\'], [\'joy\'], [\'anger\', \'surprise\'], [\'anger\', \'surprise\'], [\'joy\'], [\'joy\'], [\'joy\']]}',
 '{"list_emotion_classes": [[\'sadness\'], [\'sadness\'], [\'anger\', \'sadness\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'joy\'], [\'anger\', \'joy\'], [\'anger\', \'joy\'], [\'anger\', \'joy\'], [\'anger\'], [\'joy\'], [\'joy\']]}',
 '{"list_emotion_classes": [[\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'anger\'], [\'joy\'], [\'anger\'], [\'anger\']]}',
 '{"list_emotion_classes": [[\'joy\'], [\'joy\'], [\'joy\']]}',
 '{"list_emotion_classes": [[\'sadness\'], [\'anger\', \'surprise\'], [\'sadn

In [78]:
grounds = df_test.emotion_c.tolist()

In [79]:
len(grounds)

156

In [80]:
import json

In [81]:
predictions = [json_repair.loads(e) for e in raw_outputs]

In [82]:
predictions

[{'list_emotion_classes': [['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy']]},
 {'list_emotion_classes': [['joy'],
   ['joy'],
   ['anger', 'surprise'],
   ['anger', 'surprise'],
   ['joy'],
   ['joy'],
   ['joy']]},
 {'list_emotion_classes': [['sadness'],
   ['sadness'],
   ['anger', 'sadness'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['joy'],
   ['anger', 'joy'],
   ['anger', 'joy'],
   ['anger', 'joy'],
   ['anger'],
   ['joy'],
   ['joy']]},
 {'list_emotion_classes': [['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['anger'],
   ['joy'],
   ['anger'],
   ['anger']]},
 {'list_emotion_classes': [['joy'], ['joy'], ['joy']]},
 {'list_emotion_classes': [['sadness'],
   ['anger', 'surprise

In [83]:
# #predictions = [e.split('\n\n')[0] for e in raw_outputs]
# bad_idx = []
# predictions = []

# for idx, e in enumerate(raw_outputs):
#     try:
#         predictions.append(json.loads(e))
#     except:
#         print(idx)
#         bad_idx.append(idx)
        


In [84]:
#len(predictions)

In [85]:
#predictions

In [86]:
#predictions = [json_repair.loads(e) for e in predictions]

In [87]:
#predictions

In [88]:
preds_l = []
bad_idx = []

for i, pred in enumerate(predictions):
    try:        
        preds_l.append(pred['list_emotion_classes'])
    except:
        print(i)
        bad_idx.append(i)

In [89]:
preds_l

[[['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy']],
 [['joy'],
  ['joy'],
  ['anger', 'surprise'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy']],
 [['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger', 'joy'],
  ['anger', 'joy'],
  ['anger', 'joy'],
  ['anger'],
  ['joy'],
  ['joy']],
 [['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['joy'],
  ['anger'],
  ['anger']],
 [['joy'], ['joy'], ['joy']],
 [['sadness'],
  ['anger', 'surprise'],
  ['sadness'],
  ['fear', 'surprise'],
  ['joy'],
  ['joy'],
  ['sadness'],
  ['sadness', 'surprise'],
  ['joy'],
  ['sadness'],
  ['sadness', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  [

In [90]:
grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]

In [91]:
len(grounds), len(preds_l)

(156, 156)

In [92]:
preds_l

[[['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy']],
 [['joy'],
  ['joy'],
  ['anger', 'surprise'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy']],
 [['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger', 'joy'],
  ['anger', 'joy'],
  ['anger', 'joy'],
  ['anger'],
  ['joy'],
  ['joy']],
 [['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['joy'],
  ['anger'],
  ['anger']],
 [['joy'], ['joy'], ['joy']],
 [['sadness'],
  ['anger', 'surprise'],
  ['sadness'],
  ['fear', 'surprise'],
  ['joy'],
  ['joy'],
  ['sadness'],
  ['sadness', 'surprise'],
  ['joy'],
  ['sadness'],
  ['sadness', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  [

In [93]:
grounds

["[['surprise', 'joy'], ['joy'], ['surprise', 'joy'], ['joy'], ['joy'], ['joy'], ['surprise'], ['joy'], ['joy'], ['neutral'], ['neutral']]",
 "[['neutral'], ['neutral'], ['anger', 'disgust'], ['anger', 'disgust'], ['neutral'], ['sadness'], ['sadness']]",
 "[['anger', 'sadness'], ['anger', 'sadness'], ['anger', 'sadness'], ['fear', 'surprise'], ['surprise'], ['joy'], ['anger', 'surprise'], ['joy'], ['joy'], ['joy'], ['anger'], ['anger'], ['surprise', 'joy'], ['fear', 'sadness'], ['fear', 'sadness'], ['fear', 'surprise']]",
 "[['anger', 'disgust'], ['anger', 'disgust'], ['anger', 'disgust'], ['fear', 'sadness'], ['fear', 'sadness', 'surprise'], ['sadness'], ['sadness'], ['fear', 'sadness'], ['sadness', 'surprise'], ['sadness', 'surprise'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger', 'disgust'], ['joy'], ['joy'], ['surprise', 'joy'], ['surprise', 'joy'], ['anger', 'surprise'], ['anger', 'surprise']]",
 "[['neutral'], ['joy'], ['joy']]",
 "[['neutral'], ['fear'], ['neutral'], ['ang

In [94]:
import ast

grounds = [ast.literal_eval(x) for x in grounds]

In [95]:
grounds

[[['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral']],
 [['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise']],
 [['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise'],
  ['anger', 

In [97]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, preds_l)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

5 20 19
87 15 16
113 1 41
121 19 18


In [98]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del preds_l[idx]

In [99]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in preds_l for item in sublist]

In [100]:
len(grounds), len(predictions)

(1271, 1271)

In [101]:
mlb = MultiLabelBinarizer()

In [103]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

In [104]:
y_pred_mhot.shape

(1271, 7)

In [105]:
y_pred_mhot.shape

(1271, 7)

In [106]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=mlb.classes_, digits=3))

              precision    recall  f1-score   support

       anger      0.547     0.594     0.570       448
     disgust      1.000     0.040     0.077        50
        fear      0.537     0.078     0.136       283
         joy      0.358     0.697     0.473       287
     neutral      0.238     0.053     0.087        94
     sadness      0.611     0.210     0.313       328
    surprise      0.536     0.282     0.370       347

   micro avg      0.471     0.360     0.408      1837
   macro avg      0.547     0.279     0.289      1837
weighted avg      0.522     0.360     0.366      1837
 samples avg      0.469     0.390     0.409      1837

